In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from xgboost import plot_importance
from sklearn.preprocessing import PolynomialFeatures
%matplotlib inline

C:\Users\59146\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
input_train=pd.read_csv("train.csv",encoding="gb2312")
input_test=pd.read_csv("test.csv",encoding="gb2312")
example=pd.read_csv("submit_example.csv",encoding="gb2312")

In [3]:
df1_train=input_train.iloc[1:,[x for x in range(23)]+[23]]
df2_train=input_train.iloc[1:,[x for x in range(23)]+[24]]
df3_train=input_train.iloc[1:,[x for x in range(23)]+[25]]
df4_train=input_train.iloc[1:,[x for x in range(23)]+[26]]

In [4]:
poly = PolynomialFeatures(degree=3)
np_train = poly.fit_transform(df1_train.iloc[:,[1,2,3,4,5,8]])
df_new_features=pd.DataFrame(np_train)
df_new_features.head()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,1.0,586.40002,187.129,1859.5271,660.37347,2997.8721,576.60004,343864.983456,109732.449343,1.090427e+06,...,2.879843e+08,1.307351e+09,2.514513e+08,5.934933e+09,1.141504e+09,2.195528e+08,2.694259e+10,5.182041e+09,9.966954e+08,1.917008e+08
1,1.0,586.40002,187.129,1859.0161,662.54938,2997.8721,576.60004,343864.983456,109732.449343,1.090127e+06,...,2.908404e+08,1.315981e+09,2.531111e+08,5.954488e+09,1.145265e+09,2.202762e+08,2.694259e+10,5.182041e+09,9.966954e+08,1.917008e+08
2,1.0,586.40002,187.129,1859.0161,662.54938,2997.8721,576.60004,343864.983456,109732.449343,1.090127e+06,...,2.908404e+08,1.315981e+09,2.531111e+08,5.954488e+09,1.145265e+09,2.202762e+08,2.694259e+10,5.182041e+09,9.966954e+08,1.917008e+08
3,1.0,586.40002,187.129,1858.8391,662.36023,2998.6790,576.60004,343864.983456,109732.449343,1.090023e+06,...,2.905914e+08,1.315584e+09,2.529666e+08,5.955993e+09,1.145246e+09,2.202133e+08,2.696435e+10,5.184831e+09,9.969636e+08,1.917008e+08
4,1.0,586.40002,187.129,1858.8391,662.36023,2998.6790,576.60004,343864.983456,109732.449343,1.090023e+06,...,2.905914e+08,1.315584e+09,2.529666e+08,5.955993e+09,1.145246e+09,2.202133e+08,2.696435e+10,5.184831e+09,9.969636e+08,1.917008e+08


In [5]:
df_train = pd.concat([pd.DataFrame(df1_train.iloc[:,:23].values),df_new_features.iloc[:,7:]],axis=1)
df_train.head()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,14001,586.40002,187.129,1859.5271,660.37347,2997.8721,348.90002,38.088829,576.60004,261,...,2.879843e+08,1.307351e+09,2.514513e+08,5.934933e+09,1.141504e+09,2.195528e+08,2.694259e+10,5.182041e+09,9.966954e+08,1.917008e+08
1,14002,586.40002,187.129,1859.0161,662.54938,2997.8721,348.90002,38.088829,576.60004,261,...,2.908404e+08,1.315981e+09,2.531111e+08,5.954488e+09,1.145265e+09,2.202762e+08,2.694259e+10,5.182041e+09,9.966954e+08,1.917008e+08
2,14003,586.40002,187.129,1859.0161,662.54938,2997.8721,348.90002,38.088829,576.60004,261,...,2.908404e+08,1.315981e+09,2.531111e+08,5.954488e+09,1.145265e+09,2.202762e+08,2.694259e+10,5.182041e+09,9.966954e+08,1.917008e+08
3,14004,586.40002,187.129,1858.8391,662.36023,2998.679,348.90002,38.088829,576.60004,261,...,2.905914e+08,1.315584e+09,2.529666e+08,5.955993e+09,1.145246e+09,2.202133e+08,2.696435e+10,5.184831e+09,9.969636e+08,1.917008e+08
4,14005,586.40002,187.129,1858.8391,662.36023,2998.679,348.90002,38.088829,576.60004,261,...,2.905914e+08,1.315584e+09,2.529666e+08,5.955993e+09,1.145246e+09,2.202133e+08,2.696435e+10,5.184831e+09,9.969636e+08,1.917008e+08


In [6]:
df_test_value=input_test.iloc[1:,:]
np_test = poly.fit_transform(df_test_value.iloc[:,[1,2,3,4,5,8]])
df_test = pd.concat([pd.DataFrame(df_test_value.iloc[:,:].values),pd.DataFrame(np_test).iloc[:,7:]],axis=1)
df_test.head()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,1,600,203.52461,2264.4098,760.75397,2998.3782,363.5,43.312592,590.60004,293.90002,...,4.402838e+08,1.735301e+09,3.418078e+08,6.839385e+09,1.347175e+09,2.653574e+08,2.695624e+10,5.309655e+09,1.045860e+09,2.060063e+08
1,2,600,203.52461,2264.4098,760.75397,2998.3782,363.5,43.312592,590.5,293.90002,...,4.402838e+08,1.735301e+09,3.417499e+08,6.839385e+09,1.346947e+09,2.652675e+08,2.695624e+10,5.308756e+09,1.045505e+09,2.059016e+08
2,3,600,203.4463,2268.2728,760.75397,2998.3782,363.5,43.312592,590.5,293.90002,...,4.402838e+08,1.735301e+09,3.417499e+08,6.839385e+09,1.346947e+09,2.652675e+08,2.695624e+10,5.308756e+09,1.045505e+09,2.059016e+08
3,4,600,203.4463,2268.2728,760.75397,2998.3782,363.5,43.312592,590.5,293.90002,...,4.402838e+08,1.735301e+09,3.417499e+08,6.839385e+09,1.346947e+09,2.652675e+08,2.695624e+10,5.308756e+09,1.045505e+09,2.059016e+08
4,5,600,203.3847,2274.1449,759.99713,2998.3782,363.5,43.312592,590.5,293.90002,...,4.389710e+08,1.731850e+09,3.410702e+08,6.832581e+09,1.345607e+09,2.650036e+08,2.695624e+10,5.308756e+09,1.045505e+09,2.059016e+08


In [7]:
# cv_params_1 = {'colsample_bylevel':[0.1,0.3,0.5,0.8,1.0]}
# other_params_1={'learning_rate':0.05,'n_estimators':1100,'n_jobs':-1,'max_depth':3,'colsample_bytree': 0.2,'subsample': 0.7,'min_child_weight': 2}
# model_1 = xgb.XGBRegressor(**other_params_1)
# optimized_GBM_1 = GridSearchCV(estimator=model_1, param_grid=cv_params_1, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

In [8]:
# optimized_GBM_1.fit(df_train.iloc[:,1:].values,df1_train.iloc[:,-1].values)
# evalute_result_1 = optimized_GBM_1.grid_scores_
# print('每轮迭代运行结果:{0}'.format(evalute_result_1))
# print('参数的最佳取值：{0}'.format(optimized_GBM_1.best_params_))
# print('最佳模型得分:{0}'.format(optimized_GBM_1.best_score_))

In [9]:
cv_params_2 = {'min_child_weight':[0.1,0.5,1,1.5,2,2.5,3]}
other_params_2={'learning_rate': 0.1,'n_estimators':1100,'max_depth':7,'n_jobs':-1,'min_child_weight':2.5,'colsample_bytree':0.2,'subsample':1}
model_2 = xgb.XGBRegressor(**other_params_2)
optimized_GBM_2 = GridSearchCV(estimator=model_2, param_grid=cv_params_2, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

In [10]:
optimized_GBM_2.fit(df_train.iloc[:,1:].values,df2_train.iloc[:,-1].values)
evalute_result_2 = optimized_GBM_2.grid_scores_
print('每轮迭代运行结果:{0}'.format(evalute_result_2))
print('参数的最佳取值：{0}'.format(optimized_GBM_2.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM_2.best_score_))

每轮迭代运行结果:[mean: -2688.38106, std: 3228.86445, params: {'min_child_weight': 0.1}, mean: -2688.38106, std: 3228.86445, params: {'min_child_weight': 0.5}, mean: -2688.38106, std: 3228.86445, params: {'min_child_weight': 1}, mean: -2688.03581, std: 3230.12532, params: {'min_child_weight': 1.5}, mean: -2688.03581, std: 3230.12532, params: {'min_child_weight': 2}, mean: -2686.96011, std: 3227.85577, params: {'min_child_weight': 2.5}, mean: -2686.96011, std: 3227.85577, params: {'min_child_weight': 3}]
参数的最佳取值：{'min_child_weight': 2.5}
最佳模型得分:-2686.9601080415746


C:\Users\59146\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:761: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [ ]:
# cv_params_3 = {'learning_rate':[0.01,0.05,0.1,0.15,0.2,0.25,0.3]}
# other_params_3={'learning_rate': 0.15,'max_depth':2,'n_estimators':1100,'n_jobs':-1,'colsample_bytree':0.1,'subsample':0.6,'min_child_weight':1.5}
# model_3 = xgb.XGBRegressor(**other_params_3)
# optimized_GBM_3 = GridSearchCV(estimator=model_3, param_grid=cv_params_3, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

In [ ]:
# optimized_GBM_3.fit(df_train.iloc[:,1:].values,df3_train.iloc[:,-1].values)
# evalute_result_3 = optimized_GBM_3.grid_scores_
# print('每轮迭代运行结果:{0}'.format(evalute_result_3))
# print('参数的最佳取值：{0}'.format(optimized_GBM_3.best_params_))
# print('最佳模型得分:{0}'.format(optimized_GBM_3.best_score_))

In [ ]:
# cv_params_4 = {'learning_rate':[0.5,0.1,0.15,0.2,0.25]}
# other_params_4={'learning_rate': 0.1,'max_depth':7,'n_estimators':1100,'n_jobs':-1,'colsample_bytree':0.2,'subsample':0.7,'min_child_weight':1}
# model_4 = xgb.XGBRegressor(**other_params_4)
# optimized_GBM_4 = GridSearchCV(estimator=model_4, param_grid=cv_params_4, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)

In [ ]:
# optimized_GBM_4.fit(df_train.iloc[:,1:].values,df4_train.iloc[:,-1].values)
# evalute_result_4 = optimized_GBM_4.grid_scores_
# print('每轮迭代运行结果:{0}'.format(evalute_result_4))
# print('参数的最佳取值：{0}'.format(optimized_GBM_4.best_params_))
# print('最佳模型得分:{0}'.format(optimized_GBM_4.best_score_))